In [139]:
from webdriver_manager.chrome import ChromeDriverManager
ChromeDriverManager().install()



'/Users/csr/.wdm/drivers/chromedriver/mac64/124.0.6367.60/chromedriver-mac-x64/chromedriver'

In [140]:
from selenium import webdriver
browser = webdriver.Chrome()

In [141]:
from selenium.webdriver.common.by import By
url = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'
browser.get(url)

In [ ]:
# 1페이지 링크 데이터 전부 수집
##한개의 베스트 셀러 링크 수집
browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href')
datas = browser.find_elements(By.CLASS_NAME, 'gd_name')
for i in datas:
    print(i.get_attribute('href'))
#<a class="gd_name" href="/Product/Goods/124999476" onclick="wiseLogV2('BS', '001_005_001', ''); ">불변의 법칙</a>
#<a class="gd_name" href="/Product/Goods/126010792" onclick="wiseLogV2('BS', '001_005_001', ''); ">나는 읽고 쓰고 버린다</a>

In [70]:
# 3페이지까지의 링크 데이터 수집
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24
# https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=3&pageSize=24
import time

link_list = []
for i in range(1,4):
    print("*" * 10, f"현재 {i} 페이지 수집 중 입니다.","*" * 10)
    url = f'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber={i}&pageSize=24'
    browser.get(url)
    ### 1페이지 전체의 링크 데이터
    browser.find_element(By.CLASS_NAME, 'gd_name').get_attribute('href') # element 요소
    datas = browser.find_elements(By.CLASS_NAME, 'gd_name') # elements : 리스트
    
    for i in datas:
        link = i.get_attribute('href')
        link_list.append(link)
        
    time.sleep(3)
    
print(link_list)

********** 현재 1 페이지 수집 중 입니다. **********
********** 현재 2 페이지 수집 중 입니다. **********
********** 현재 3 페이지 수집 중 입니다. **********
['https://www.yes24.com/Product/Goods/126010792', 'https://www.yes24.com/Product/Goods/124999476', 'https://www.yes24.com/Product/Goods/126029503', 'https://www.yes24.com/Product/Goods/125295101', 'https://www.yes24.com/Product/Goods/125992096', 'https://www.yes24.com/Product/Goods/125491624', 'https://www.yes24.com/Product/Goods/125651056', 'https://www.yes24.com/Product/Goods/126110769', 'https://www.yes24.com/Product/Goods/117014613', 'https://www.yes24.com/Product/Goods/125576919', 'https://www.yes24.com/Product/Goods/123675187', 'https://www.yes24.com/Product/Goods/122120495', 'https://www.yes24.com/Product/Goods/126029190', 'https://www.yes24.com/Product/Goods/125973831', 'https://www.yes24.com/Product/Goods/90926731', 'https://www.yes24.com/Product/Goods/123400303', 'https://www.yes24.com/Product/Goods/96547408', 'https://www.yes24.com/Product/Goods/12601200

In [ ]:

# 상세 페이지로 이동
# for link in link_list:
# #     # 상세 페이지로 이동
#      browser.get(link)
import re 
browser.get(link_list[7])

title = browser.find_element(By.CLASS_NAME, 'gd_name').text
author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
rating = browser.find_element(By.CLASS_NAME, 'gd_lnkRate').text
rating = re.sub("[ㄱ-힣]","",rating).strip()
review = browser.find_element(By.CLASS_NAME, 'txC_blue').text
sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
ranking = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[0]
ranking_weeks = browser.find_element(By.CLASS_NAME, 'gd_best').text.split(" | ")[1]
title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks

In [ ]:
# 데이터베이스 연동 후 -> 수정된 데이터를 DB에 저장
import pymysql #pip install pymysql
import time
import re
from datetime import datetime
conn = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = '4682',
    db = 'yes24',
    charset = 'utf8mb4',
    cursorclass=pymysql.cursors.DictCursor
)

with conn.cursor() as cur:
    for link in link_list:
        browser.get(link)
        
        title = browser.find_element(By.CLASS_NAME, 'gd_name').text
        author = browser.find_element(By.CLASS_NAME, 'gd_auth').text
        publisher = browser.find_element(By.CLASS_NAME, 'gd_pub').text
        
        #2023년 12월 25일 -> 2023-12-25
        publishing = browser.find_element(By.CLASS_NAME, 'gd_date').text
        
        match = re.search(r'(\d+)년 (\d+)월 (\d+)일', publishing)
        
        if match:
            year, month, day = match.groups()
            data_obj = datetime(int(year), int(month), int(day)) 
            publishing = data_obj.strftime("%Y-%m-%d")
        else:
            publishing = "2023-01-01"
        # 가끔 평점이 없을 경우"yes_b" 클래스를 가져오다보면 아래 책 미리보기에서 yes_b를 가져오는 에러가 발생해 
        # 'gd_lnkRate'를 가져와 한글과 숫자(평점9.7) 을 한번에 가져와 한글을 벗겨 적용
        try:
            rating = browser.find_element(By.CLASS_NAME, 'gd_lnkRate').text
            rating = float(re.sub("[ㄱ-힣]","",rating).strip())
        except:
            rating = None
            
        review = browser.find_element(By.CLASS_NAME, 'txC_blue').text.replace(",","")
        del match
        match = re.search(r"\d+", review)
        if match:
            review = int(match.group())
        else:
            review = None
        sales = browser.find_element(By.CLASS_NAME, 'gd_sellNum').text.split(" ")[2]
        sales = int(sales.replace(",", ""))
        
        price = browser.find_element(By.CLASS_NAME, 'yes_m').text[:-1]
        price = int(price.replace(",", ""))
        
        # 국내도서 랭킹과 주간 랭킹 중 한개만 포함된 경우 국내도서 랭킹 표시
        full_text = browser.find_element(By.CLASS_NAME, 'gd_best').text
        print('full_text : ' , full_text) 
        parts = full_text.split(" | ")
        if len(parts) == 0:
            ranking = 0
            ranking_weeks = 0
        else:
            try:
                ranking = parts[0]
                
            except:
                ranking = 0
                
            try:
                ranking_weeks = parts[1]
                
            except:
                ranking_weeks = 0
                                    
        sql = """INSERT INTO Books(
            title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks
            )
            VALUES(
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s 
            )
            """
    
        cur.execute(sql, (title, author, publisher, publishing, rating, review, sales, price, ranking, ranking_weeks))
        conn.commit()
        time.sleep(0.1)